# 🕵️‍♂️ Project Nexus: Client 360 Assistant (AgentExecutor Version)
**Module:** Agentic AI Day 2 (Tools & Mechanics)  
**Objective:** Solve the Project Nexus business case using the high-level `AgentExecutor` framework for rapid prototyping.

---

### 📋 Architecture
Instead of writing the `while` loop manually, we will use LangChain's **`AgentExecutor`**. 
This class handles:
1.  Calling the LLM.
2.  Parsing the Tool Calls.
3.  Executing the Tools.
4.  Handling Errors.
5.  Managing History (`agent_scratchpad`).

In [26]:
# CELL 1: SETUP & IMPORTS
import os
import getpass
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, StructuredTool
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# API Key Check
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

# Initialize Brain
llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("✅ Setup Complete")

✅ Setup Complete


In [27]:
# CELL 2: MOCK DATABASE
CLIENT_DB = {
    "Acme Corp": {"industry": "Retail", "contact": "Alice Smith"},
    "Globex": {"industry": "Tech", "contact": "Bob Jones"},
    "Soylent Corp": {"industry": "Food", "contact": "Carol White"}
}

### 🛠️ Step 1: Define Tools
We use the same robust tool definitions as the manual version.

In [28]:
# CELL 3: TOOL DEFINITIONS

# --- TOOL A: RETRIEVAL ---
@tool
def get_client_details(client_name: str) -> str:
    """Retrieves industry and key contact for a client.
    Returns an error string if client is not found."""
    
    for key, value in CLIENT_DB.items():
        if key.lower() == client_name.lower():
            return f"Client: {key} | Industry: {value['industry']} | Contact: {value['contact']}"
            
    return f"Error: Client '{client_name}' not found in the database."

# --- TOOL B: SCHEDULER (With Pydantic) ---
class ScheduleInput(BaseModel):
    client_name: str = Field(description="Name of the client")
    date: str = Field(description="Date of the meeting (e.g., 'Oct 20th')")
    duration_minutes: int = Field(description="Duration in minutes (must be an integer)")

def schedule_meeting_func(client_name: str, date: str, duration_minutes: int) -> str:
    """Schedules a meeting in the calendar."""
    return f"SUCCESS: Meeting scheduled with {client_name} on {date} for {duration_minutes} minutes."

schedule_tool = StructuredTool.from_function(
    func=schedule_meeting_func,
    name="schedule_meeting",
    description="Schedule a meeting with a client.",
    args_schema=ScheduleInput
)

# List of tools
tools = [get_client_details, schedule_tool]
print("✅ Tools Ready")

✅ Tools Ready


### 🤖 Step 2: Create the AgentExecutor
Here we construct the "Easy Button". Note that `AgentExecutor` requires a prompt with a placeholder for `{agent_scratchpad}`.

In [29]:
# CELL 4: AGENT CONSTRUCTION
from langchain.agents import create_agent

# 1. Define the Prompt - create_agent handles this internally
# 2. Create and execute the agent
nexus_agent = create_agent(llm, tools, debug=True)

print("✅ Agent Ready")

✅ Agent Ready


### 🧪 Step 3: Execution
Let's test the agent against the requirements.

In [30]:
# SCENARIO 1: Composite Task (Lookup + Act)
query = "Find the contact for Acme Corp and book a 45 min meeting with them on Oct 20th."

print(f"🚀 EXECUTING: '{query}'")
response = nexus_agent.invoke({"messages": [("user", query)]})

print("-" * 50)
print(f"🏁 FINAL OUTPUT: {response['messages'][-1].content}")

🚀 EXECUTING: 'Find the contact for Acme Corp and book a 45 min meeting with them on Oct 20th.'
[values] {'messages': [HumanMessage(content='Find the contact for Acme Corp and book a 45 min meeting with them on Oct 20th.', additional_kwargs={}, response_metadata={}, id='05b4d663-5b93-4ffe-a3cc-62b43ffb6342')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 141, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_689bad8e9a', 'id': 'chatcmpl-CgW6O6CEHSxTcWStnByOC1ayE7n0z', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f9d80f13-a678-4619-9a4c-bd1f926e5036-0', tool_ca

In [31]:
# SCENARIO 2: Error Handling (Missing Client)
query = "Book a meeting with Nvidia."

response = nexus_agent.invoke({"messages": [("user", query)]})

[values] {'messages': [HumanMessage(content='Book a meeting with Nvidia.', additional_kwargs={}, response_metadata={}, id='d244f8c0-7cda-4530-9934-7ec162f8763e')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 125, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_689bad8e9a', 'id': 'chatcmpl-CgW6TintJldHOP5vgw9zlJuNB5FMa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--a8a30c0f-8e93-4ca2-b406-5c22d0e03bd4-0', tool_calls=[{'name': 'get_client_details', 'args': {'client_name': 'Nvidia'}, 'id': 'call_w39V64456iyz2VlKQPIksxLQ', 'type': 'tool_call'}], usage_metadata